# Zadanie 1

In [8]:
def pl_true_symbols(s, m):
    if isinstance(s, str):  
        if s in m:
            return m[s]
        else:
            raise ValueError(f"Symbol '{s}' nie znaleziony w przypisaniu m.")

    elif isinstance(s, tuple):  
        operator = s[0] 

        if operator == '~': # Negacja
            if len(s) != 2:
                raise ValueError(f"Operator ~ oczekuje jednego argumentu, otrzymano {len(s)-1} dla {s}")
            val = pl_true_symbols(s[1], m)
            return 1 - val

        elif operator == '&': # Koniunkcja (AND)
            if len(s) != 3:
                raise ValueError(f"Operator & oczekuje dwóch argumentów, otrzymano {len(s)-1} dla {s}")
            val1 = pl_true_symbols(s[1], m)
            val2 = pl_true_symbols(s[2], m)
            return 1 if val1 == 1 and val2 == 1 else 0

        elif operator == '|': # Alternatywa (OR)
            if len(s) != 3:
                raise ValueError(f"Operator | oczekuje dwóch argumentów, otrzymano {len(s)-1} dla {s}")
            val1 = pl_true_symbols(s[1], m)
            val2 = pl_true_symbols(s[2], m)
            return 1 if val1 == 1 or val2 == 1 else 0

        elif operator == '->': # Implikacja
            if len(s) != 3:
                raise ValueError(f"Operator -> oczekuje dwóch argumentów, otrzymano {len(s)-1} dla {s}")
            val1 = pl_true_symbols(s[1], m) 
            val2 = pl_true_symbols(s[2], m)
            
            # A -> B jest równoważne ~A | B
            return 1 if (1 - val1) == 1 or val2 == 1 else 0


        else:
            raise ValueError(f"Nieznany operator: {operator} w zdaniu {s}")
    else:
        raise TypeError(f"Nieprawidłowy format zdania S: {s}. Oczekiwano stringa lub krotki.")



### Przypadek 0: 0 spójników

In [9]:
m0 = {'p': 1}
s0 = 'p'
print(f"Zdanie: {s0}, Przypisanie: {m0}, Wynik: {pl_true_symbols(s0, m0)}")

Zdanie: p, Przypisanie: {'p': 1}, Wynik: 1


### Przypadek 1: 1 spójnik

In [10]:
m1a = {'p': 0}
s1a = ('~', 'p')
print(f"Zdanie: {s1a}, Przypisanie: {m1a}, Wynik: {pl_true_symbols(s1a, m1a)}")

Zdanie: ('~', 'p'), Przypisanie: {'p': 0}, Wynik: 1


In [11]:
m1b = {'p': 1, 'q': 1}
s1b = ('&', 'p', 'q')
print(f"Zdanie: {s1b}, Przypisanie: {m1b}, Wynik: {pl_true_symbols(s1b, m1b)}")

Zdanie: ('&', 'p', 'q'), Przypisanie: {'p': 1, 'q': 1}, Wynik: 1


### Przypadek 2: 2 spójniki

In [12]:
m2 = {'p': 0, 'q': 1, 'r': 0} 
s2 = ('->', ('|', 'p', ('~', 'q')), 'r')
print(f"Zdanie: {s2}, Przypisanie: {m2}, Wynik: {pl_true_symbols(s2, m2)}")

Zdanie: ('->', ('|', 'p', ('~', 'q')), 'r'), Przypisanie: {'p': 0, 'q': 1, 'r': 0}, Wynik: 1


In [13]:
m2_v2 = {'p': 1, 'q': 0, 'r': 0} 
s2_v2 = ('->', ('&', 'p', ('~', 'q')), 'r')
print(f"Zdanie: {s2_v2}, Przypisanie: {m2_v2}, Wynik: {pl_true_symbols(s2_v2, m2_v2)}")

Zdanie: ('->', ('&', 'p', ('~', 'q')), 'r'), Przypisanie: {'p': 1, 'q': 0, 'r': 0}, Wynik: 0


# Zadanie 2

# Zadanie 9

Oznaczenia (predykaty i stałe):

  Loves(x,y): x kocha y

  Kills(x,y): x zabija y

  Animal(y): y jest zwierzęciem

  Tuna: kot Tuna (stała)

  Jack, Jola: osoby (stałe)

  ∀, ∃: kwantyfikatory ogólny i szczegółowy
    
Wiedza ogólna (KB):
1. Każdy, kto kocha wszystkie zwierzęta, jest przez kogoś kochany.
    ∀x[(∀y(Animal(y)→Loves(x,y)))→(∃zLoves(z,x))]
2. Nikt nie kocha tego, kto zabija zwierzę.
    ∀x∀y∀z[(Animal(z)∧Kills(y,z))→¬Loves(x,y)]
3. Jack kocha wszystkie zwierzęta.
    ∀y(Animal(y)→Loves(Jack,y))
4. Albo Jack, albo Jola zabili kota Tuna.
    Kills(Jack,Tuna)∨Kills(Jola,Tuna)
5. Tuna jest zwierzęciem.
    Animal(Tuna)



In [7]:
from sympy import symbols, Or, And, Not
from sympy.logic.boolalg import to_cnf
from sympy.logic.inference import satisfiable

# Predykaty jako funkcje
Loves = lambda x, y: symbols(f"Loves_{x}_{y}")
Kills = lambda x, y: symbols(f"Kills_{x}_{y}")
Animal = lambda x: symbols(f"Animal_{x}")

# Stałe (osoby i kot)
Jack = "Jack"
Jola = "Jola"
Tuna = "Tuna"

# KB: baza wiedzy w postaci CNF, po ręcznej skolemizacji

kb = And(
    # (1) Jack kocha wszystkie zwierzęta → ¬Animal(Tuna) ∨ Loves(Jack, Tuna)
    Or(Not(Animal(Tuna)), Loves(Jack, Tuna)),

    # (2) Tuna jest zwierzęciem
    Animal(Tuna),

    # (3) Jack lub Jola zabili kota Tuna
    Or(Kills(Jack, Tuna), Kills(Jola, Tuna)),

    # (4) Nikt nie kocha tego, kto zabija zwierzę → rozpisane dla Jacka i Joli
    Or(Not(Animal(Tuna)), Not(Kills(Jack, Tuna)), Not(Loves("Someone", Jack))),
    Or(Not(Animal(Tuna)), Not(Kills(Jola, Tuna)), Not(Loves("Someone", Jola))),

    # (5) Każdy, kto kocha wszystkie zwierzęta, jest przez kogoś kochany
    # Rozpisane dla Jacka i Tuna (skolemizowane)
    Or(Animal(Tuna), Loves("f", Jack)),
    Or(Not(Loves(Jack, Tuna)), Loves("f", Jack))
)

# Negacja wniosku: zakładamy, że Jola NIE zabiła Tuny
negated_conclusion = Not(Kills(Jola, Tuna))

# Łączymy bazę wiedzy z negacją wniosku
full_formula = And(kb, negated_conclusion)

# Przekształcamy do CNF (można też wypisać: print(to_cnf(full_formula, simplify=True)))
cnf = to_cnf(full_formula, simplify=True)

# Sprawdzamy, czy formuła jest spełnialna
result = satisfiable(cnf)

# Interpretacja wyniku
if not result:
    print("Wniosek: Z KB wynika, że Jola zabiła kota Tuna.")
else:
    print("Nie można jednoznacznie stwierdzić, że Jola zabiła kota Tuna.")
    print("Spełniające przypisanie:", result)


Nie można jednoznacznie stwierdzić, że Jola zabiła kota Tuna.
Spełniające przypisanie: {Animal_Tuna: True, Kills_Jola_Tuna: False, Loves_Jack_Tuna: True, Loves_f_Jack: True, Kills_Jack_Tuna: True, Loves_Someone_Jack: False}
